In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
df1 = pd.read_excel('./data/아모레G_시세추이.xlsx')

C:\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# ATR(Average True Range) 변동성 지표 생성

df_atr = df1.iloc[:,[0,1,5,6]]
df_atr.columns = ['date', 'fin', 'high', 'low']

In [4]:
df_atr

,date,fin,high,low
0,2021/08/31,57800,58300,55600
1,2021/08/30,56200,57200,56200
2,2021/08/27,56800,56900,55800
3,2021/08/26,56300,57500,56000
4,2021/08/25,56700,57500,56100
...,...,...,...,...
2460,2011/09/06,234500,235500,229500
2461,2011/09/05,233000,234000,226500
2462,2011/09/02,231000,237500,230500
2463,2011/09/01,234000,241000,225500


In [5]:
lst = []
atr = []

for i in range(0,len(df_atr)-1):
    a = df_atr.iloc[i,2] - df_atr.iloc[i,3]
    b = df_atr.iloc[i,2] - df_atr.iloc[i+1,1]
    c = df_atr.iloc[i,3] - df_atr.iloc[i+1,1]
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))

In [6]:
# y값(종가 데이터) 준비

fin = list(df_atr[0:-1]['fin'])
del fin[1556:1566] # 거래정지 시점 데이터 행 삭제

In [7]:
# 거래량 데이터 추출

df1 = df1.drop(len(df1)-1,axis=0)
df1 = df1[['일자', '거래량']]

In [8]:
# PER, PBR 추출

df2 = pd.read_excel('./data/아모레G_PER.xlsx')
df2 = df2[['일자', 'PER', 'PBR']]

In [9]:
# 거래주체별 순매수 데이터 추출

df3 = pd.read_excel('./data/아모레G_거래실적.xlsx')
df3 = df3.drop(columns=['전체'], axis=1)

In [10]:
# 데이터 병합
amore_G = pd.merge(df1, df2, on='일자', how='outer')
amore_G = pd.merge(amore_G, df3, on='일자', how='outer')
amore_G['ATR'] = atr

In [11]:
# 액면분할에 따른 전처리
amore_G = amore_G.drop(index = [1556,1557,1558,1559,1560,1561,1562,1563,1564,1565], axis=1)
amore_G.iloc[1556:,8] = amore_G.iloc[1556:,8]/10
amore_G.iloc[1556:,1] = amore_G.iloc[1556:,1]*10

In [12]:
# 날짜 데이터로 변환
amore_G['일자'] = pd.to_datetime(amore_G['일자'])
amore_G.rename(columns={'일자':'DATE'}, inplace=True)

In [13]:
# 거시경제 지표와 병합
macro_data = pd.read_csv('./data/merge_data.csv')
macro_data['DATE'] = pd.to_datetime(macro_data['DATE'])    # 날짜 데이터로 변환
macro_data = macro_data.dropna(axis=0)                     # 결측치가 포함된 행 제거
df_amore_G = pd.merge(amore_G, macro_data, how='left', on='DATE')

In [15]:
df_amore_G

,DATE,거래량,PER,PBR,기관 합계,기타법인,개인,외국인 합계,ATR,NASDAQ,S&P,CBOE,Exchange rate,futures2y,futures10y
0,2021-08-31,304205,781.08,1.71,-1.255206e+08,30276000.0,9.388480e+07,1.359800e+06,2700.0,"15,259.20","4,522.68",16.48,"1,158.73",110.32,134.05
1,2021-08-30,152944,759.46,1.66,-3.217738e+08,1970500.0,2.800442e+09,-2.480639e+09,1000.0,"15,265.90","4,528.79",16.19,"1,164.34",110.32,134.13
2,2021-08-27,99603,767.57,1.68,1.154188e+09,-283000.0,2.906114e+08,-1.444517e+09,1100.0,"15,129.50","4,509.37",16.39,"1,161.23",110.30,133.89
3,2021-08-26,104525,760.81,1.66,-3.058674e+08,-19187500.0,2.116813e+09,-1.791758e+09,1500.0,"14,945.80","4,469.91",18.84,"1,170.53",110.25,133.59
4,2021-08-25,101597,766.22,1.67,2.275278e+08,-10054100.0,1.030325e+09,-1.247798e+09,1400.0,"15,041.90","4,496.19",16.79,"1,163.87",110.25,133.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,2011-09-07,35540,16.94,1.18,3.029710e+08,-4998000.0,3.219350e+07,-3.301665e+08,1050.0,"2,548.90","1,198.62",33.38,"1,069.65",110.27,130.89
2450,2011-09-06,45910,17.16,1.20,9.045400e+07,0.0,-5.351450e+07,-3.693950e+07,600.0,"2,473.80","1,165.24",37.00,"1,074.65",110.27,131.22
2451,2011-09-05,36210,17.05,1.19,-3.927650e+07,0.0,5.751450e+07,-1.823800e+07,750.0,NaN,NaN,NaN,NaN,NaN,NaN
2452,2011-09-02,38310,16.91,1.18,-2.088455e+08,0.0,1.046165e+08,1.042290e+08,700.0,"2,480.30","1,173.97",33.92,"1,065.60",110.25,131.30


In [17]:
# y값(종가 데이터) 병합
df_amore_G['y'] = fin

In [18]:
# 종가 열 액면분할 반영
df_amore_G.iloc[1556:,15] = df_amore_G.iloc[1556:,15]/10

In [19]:
# 결측치를 전일 값으로 대체
df_amore_G.sort_values('DATE', inplace=True)
df_amore_G.fillna(method='ffill', inplace=True)

In [21]:
df_amore_G.to_excel('./data/amore_G.xlsx',index=False)

In [22]:
amore_G_df = pd.read_excel('./data/amore_G.xlsx')

In [23]:
amore_G_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2454 entries, 0 to 2453
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATE           2454 non-null   datetime64[ns]
 1   거래량            2454 non-null   int64         
 2   PER            2454 non-null   float64       
 3   PBR            2454 non-null   float64       
 4   기관 합계          2454 non-null   int64         
 5   기타법인           2454 non-null   int64         
 6   개인             2454 non-null   int64         
 7   외국인 합계         2454 non-null   int64         
 8   ATR            2454 non-null   int64         
 9   NASDAQ         2454 non-null   object        
 10  S&P            2454 non-null   object        
 11  CBOE           2454 non-null   float64       
 12  Exchange rate  2454 non-null   object        
 13  futures2y      2454 non-null   float64       
 14  futures10y     2454 non-null   float64       
 15  y              2454 n